In [ ]:
import findspark

findspark.init('/Users/hyunseokjung/spark-3.3.0')

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10-assembly_2.12-3.3.1 pyspark-shell'

import sys
import time
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

n_secs = 1
topic = 'pyspark-kafka-demo'

conf = SparkConf().setAppName('KafkaStreamProcessor').setMaster('local[*]')
sc = SparkContext(conf=conf)
sc.setLogLevel('WARN')
ssc = StreamingContext(sc, n_secs)

kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
    'bootstrap.servers': 'localhost:9092',
    'group.id':'video-group',
    'fetch.message.max.bytes':'15728640',
    'auto.offset.reset':'largest'
})

lines = kafkaStream.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b : a+b)
counts.pprint()

ssc.start()
time.sleep(600)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

